In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import gc
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

from importlib import reload
import date
reload(date)
from date import *
import model
reload(model)
from model import *
from metric import get_weights, NWRMSLE_log

In [2]:
df = load_data_in_date_range('./data/train_processed.csv', '2017-04-04', '2017-08-15', 110000000)

Reading batch from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 8659998, mapping...
Mapped, reducing...
Batch done.
Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [3]:
print('Reading additional datasets')
items = pd.read_csv('./data/items_encoded.csv')
stores = pd.read_csv('./data/stores_encoded.csv')

Reading additional datasets


In [4]:
print('Converting data and joining additional data')
df = convert_unit_sales(df)
df = fill_empty_sales(df)
df, cols_categories = extend_dataset(df, items, stores)
del df['id']
del df['weight']
gc.collect()

Converting data and joining additional data


219

In [5]:
print('Adding lagged features')
df_prev = df[['item_nbr', 'store_nbr', 'date', 'unit_sales']]
df, cols_lagged = fill_lagged(df, df_prev, 12, 18, True)
del df_prev
gc.collect()
df.head()

Adding lagged features


/home/ubuntu/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ubuntu/grocery/model.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_prev[colname] = df_prev['unit_sales']


Processing lagged 12...
Lagged 12 is done.
Processing lagged 13...
Lagged 13 is done.
Processing lagged 14...
Lagged 14 is done.
Processing lagged 15...
Lagged 15 is done.
Processing lagged 16...
Lagged 16 is done.
Processing lagged 17...
Lagged 17 is done.
Processing lagged 18...
Lagged 18 is done.


,date,store_nbr,item_nbr,unit_sales,onpromotion,weekday,weekend,salary,family,class,...,state,type,cluster,unit_sales(t-12),unit_sales(t-13),unit_sales(t-14),unit_sales(t-15),unit_sales(t-16),unit_sales(t-17),unit_sales(t-18)
0,1635,46,1695836,2.735787,False,5,True,False,30,2004,...,12,0,14,2.995732,3.288285,3.194839,2.603847,2.442347,3.597312,2.956699
1,1636,46,1695836,3.821825,False,6,True,False,30,2004,...,12,0,14,2.545531,2.995732,3.288285,3.194839,2.603847,2.442347,3.597312
2,1637,46,1695836,2.530094,False,0,False,False,30,2004,...,12,0,14,3.076736,2.545531,2.995732,3.288285,3.194839,2.603847,2.442347
3,1638,46,1695836,2.697052,False,1,False,False,30,2004,...,12,0,14,2.978395,3.076736,2.545531,2.995732,3.288285,3.194839,2.603847
4,1639,46,1695836,2.812786,True,2,False,False,30,2004,...,12,0,14,2.652361,2.978395,3.076736,2.545531,2.995732,3.288285,3.194839


In [6]:
df['date'] = df['date'].astype('int16', copy=False)
df['store_nbr'] = df['store_nbr'].astype('int16', copy=False)
df['item_nbr'] = df['item_nbr'].astype('int32', copy=False)
df['weekday'] = df['weekday'].astype('int16', copy=False)
df['family'] = df['family'].astype('int16', copy=False)
df['class'] = df['class'].astype('int16', copy=False)
df['perishable'] = df['perishable'].astype('bool', copy=False)
df['city'] = df['city'].astype('int16', copy=False)
df['state'] = df['state'].astype('int16', copy=False)
df['type'] = df['type'].astype('int16', copy=False)
df['cluster'] = df['cluster'].astype('int16', copy=False)
gc.collect()

0

In [ ]:
folds = [('2017-06-16', '2017-06-30', '2017-07-01', '2017-07-15'),
         ('2017-07-01', '2017-07-15', '2017-07-16', '2017-07-31'),
         ('2017-07-16', '2017-07-31', '2017-08-01', '2017-08-15')]

In [ ]:
errors = []
models = []

x_cols = cols_lagged

for train_start, train_end, test_start, test_end in folds:
    print("Extracting fold...")
    train = extract_by_date(df, train_start, train_end)
    test= extract_by_date(df, test_start, test_end)
    
    print("Preparing train...")
    train_X = train[x_cols].fillna(0.0)
    train_y = train['unit_sales']
    
    print("Preparing test...")
    test_X = test[x_cols].fillna(0.0)
    test_y = test['unit_sales']
    test_weights = get_weights(test['item_nbr'])

    print("="*80)
    model = KNeighborsRegressor(n_neighbors=10, metric='canberra', n_jobs=4)
    print("="*80)
    model.fit(train_X, train_y)
    
    test_y_pred = model.predict(test_X)
    error = NWRMSLE_log(test_y_pred, test_y, test_weights)
    print('Validation error: {}'.format(error))

    errors.append(error)
    model.append(model)

Extracting fold...
Preparing train...
Preparing test...


In [ ]:
!telegram-send "KNN validation finished. Results: $errors"